In [27]:
import pandas as pd
from pulp import *

In [28]:
df = pd.read_excel("First_Draft.xlsx")
df

,Team,Player ID,First Name,Second Name,Web Name,Position,Start Price,Current Price,Selected By,Transfers In,Transfers Out,Total Points,Bonus,Minutes,Goals Scored,Assists,Clean Sheets,Status,Form
0,Arsenal,1,Cédric,Alves Soares,Cédric,Defender,0,4.5,0.3,0,0,48,3,1481,1,1,3,a,0.0
1,Arsenal,2,Bernd,Leno,Leno,Goalkeeper,0,4.5,0.7,0,0,10,0,360,0,0,1,a,0.0
2,Arsenal,3,Granit,Xhaka,Xhaka,Midfielder,0,5.0,0.5,0,0,60,3,2327,1,2,6,a,0.0
3,Arsenal,4,Mohamed,Elneny,Elneny,Midfielder,0,4.5,1.3,0,0,27,0,801,0,2,0,a,0.0
4,Arsenal,5,Rob,Holding,Holding,Defender,0,4.5,0.2,0,0,29,2,840,1,0,2,a,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Wolves,492,Matija,Šarkić,Šarkić,Goalkeeper,0,4.0,3.2,0,0,0,0,0,0,0,0,a,0.0
532,Wolves,493,Morgan,Gibbs-White,Gibbs-White,Midfielder,0,5.5,0.2,0,0,1,0,7,0,0,0,a,0.0
533,Wolves,494,Yerson,Mosquera Valdelamar,Mosquera,Defender,0,4.0,1.2,0,0,0,0,0,0,0,0,a,0.0
534,Wolves,503,João Filipe Iria,Santos Moutinho,Moutinho,Midfielder,0,5.0,2.1,0,0,96,8,2970,2,1,11,a,0.0


In [29]:
df = df[['Web Name', 'Team', 'Position', 'Total Points', 'Current Price']]
df = df.rename(columns={'Web Name': 'Name', 'Total Points': 'Points', 'Current Price': 'Price'})

In [30]:

POS = df.Position.unique()
CLUBS = df.Team.unique()
BUDGET = 100
pos_available = {
    'Defender': 5,
    'Forward': 3,
    'Midfielder': 5,
    'Goalkeeper': 2,
}

# Initialize Variables
names = [df.Name[i] for i in df.index]
teams =  [df.Team[i] for i in df.index]
positions =  [df.Position[i] for i in df.index]
prices =  [df.Price[i] for i in df.index]
points =  [df.Points[i] for i in df.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in df.index]

In [31]:
# Initialize the problem
prob = LpProblem("FPL Player Choices", LpMaximize)

/opt/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [32]:
# Build the constraints
# Define the objective
prob += lpSum(players[i] * points[i] for i in range(len(df))) # Objective

prob += lpSum(players[i] * df.Price[df.index[i]] for i in range(len(df))) <= BUDGET # Budget Limit

for pos in POS:
  prob += lpSum(players[i] for i in range(len(df)) if positions[i] == pos) <= pos_available[pos] # Position Limit

for club in CLUBS:
  prob += lpSum(players[i] for i in range(len(df)) if teams[i] == club) <= 3 # Club Limit

In [33]:
# Solve the problem
prob.solve()

1

In [34]:
for v in prob.variables():
  if v.varValue != 0:
    name = df.Name[int(v.name.split("_")[1])]
    club = df.Team[int(v.name.split("_")[1])]
    position = df.Position[int(v.name.split("_")[1])]
    point = df.Points[int(v.name.split("_")[1])]
    price = df.Price[int(v.name.split("_")[1])]
    print(name, position, club, point, price, sep=" | ")

Saka | Midfielder | Arsenal | 179 | 8.0
Gabriel | Defender | Arsenal | 146 | 5.0
Maddison | Midfielder | Leicester City | 181 | 8.0
Van Dijk | Defender | Liverpool | 183 | 6.5
Alisson | Goalkeeper | Liverpool | 176 | 5.5
Alexander-Arnold | Defender | Liverpool | 208 | 7.5
Cancelo | Defender | Manchester City | 201 | 7.0
Bernardo | Midfielder | Manchester City | 155 | 7.0
Cash | Defender | Aston Villa | 147 | 5.0
Ward-Prowse | Midfielder | Southampton | 159 | 6.5
Antonio | Forward | West Ham | 140 | 7.5
Bowen | Midfielder | West Ham | 206 | 8.5
Sá | Goalkeeper | Wolves | 146 | 5.0
Toney | Forward | Brentford | 139 | 7.0
Mbeumo | Forward | Brentford | 119 | 6.0
